# Encryption Composers

> Located in [./lib](./lib/).

In this project, composers (loose shorthand reference), are a unique classes to this project that enable rapidly tested encryption algorithms whilst collecting useful metrics. Overtime, various forms of composers will be created. In essence, composers are high-level encryption-algorithm harnesses.

# Simple Encryption Composer

Built for a single algorithm, this harness aims to make simple usages of new algorithms to compare them to others, and test for mock-production processes.

## Fire & Forget Ideology

The idea is to fire the algorithm in the composer and forget about about logistics while you review the results.

In [ ]:
from lib.EncryptionAlgorithm import EncryptionAlgorithm, SIMPLE_COMPOSER_TYPE
from lib.SimpleEncryptionComposer import SimpleEncryptionComposer

class Aes256Algorithm(EncryptionAlgorithm):
    def __init__(self):
        super().__init__(composer_type=SIMPLE_COMPOSER_TYPE)
    
    def encrypt(self, payload):
        # Implementation for encryption
        pass

    def decrypt(self, payload):
        # Implementation for decryption
        pass

aes256_composer = SimpleEncryptionComposer()